Xception

In [1]:
import glob
import os
import cv2
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
from matplotlib import cm
from PIL import Image, ImageOps
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.callbacks import ModelCheckpoint
from keras.applications.xception import Xception
from keras import backend as K
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error
from tensorflow.python.keras.engine.sequential import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten, InputLayer, MaxPooling2D, AveragePooling2D
from tensorflow.python.keras.layers import Convolution2D
from tensorflow.python.keras import layers, models, Model
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip ./drive/MyDrive/data/mrdata.zip

Archive:  ./drive/MyDrive/data/mrdata.zip
   creating: mrdata/
   creating: mrdata/10/
   creating: mrdata/10/depth/
  inflating: mrdata/10/depth/11.29.24.441.png  
  inflating: mrdata/10/depth/11.29.24.554.png  
  inflating: mrdata/10/depth/11.29.24.663.png  
  inflating: mrdata/10/depth/11.29.24.772.png  
  inflating: mrdata/10/depth/11.29.24.883.png  
  inflating: mrdata/10/depth/11.29.24.997.png  
  inflating: mrdata/10/depth/11.29.25.104.png  
  inflating: mrdata/10/depth/11.29.25.210.png  
  inflating: mrdata/10/depth/11.29.25.322.png  
  inflating: mrdata/10/depth/11.29.25.433.png  
  inflating: mrdata/10/depth/11.29.25.543.png  
  inflating: mrdata/10/depth/11.29.25.651.png  
  inflating: mrdata/10/depth/11.29.25.764.png  
  inflating: mrdata/10/depth/mirror11.29.24.405.png  
  inflating: mrdata/10/depth/mirror11.29.24.489.png  
  inflating: mrdata/10/depth/mirror11.29.24.565.png  
  inflating: mrdata/10/depth/mirror11.29.24.646.png  
  inflating: mrdata/10/depth/mirror11.29.24

In [4]:
W = 512
H = 424
max_value_for_normalization = -1

def make_df(path_to_xlsx, path_to_folder):
    if not os.path.exists(path_to_xlsx):
        print('[Err] Fail xlsx path')
        return
    if not os.path.exists(path_to_folder):
        print('[Err] Fail folder path')
        return

    df = pd.read_excel(path_to_xlsx)
    df = df.fillna(0)
    df['Body length (cm)'] = df['Body length (cm)'].astype(int)
    df['Chest girth (cm)'] = df['Chest girth (cm)'].astype(int)
    df['Withers height (cm)'] = df['Withers height (cm)'].astype(int)
    df['Chest depth (cm)'] = df['Chest depth (cm)'].astype(int)
    df['Chest width (cm)'] = df['Chest width (cm)'].astype(int)
    df['Live weight (kg)'] = df['Live weight (kg)'].astype(int)
    df.drop(
        columns=['Frame', 'Content', 'Number of pig', 'Left', 'Right', 'Quality', 'Transform\nT_right_to_left', 'Sex', 'Age (days)'], inplace=True)

    df = df.iloc[2:]
    df['image'] = None
    for index, row in df.iterrows():
        path = f'{path_to_folder}/{index}/depth'
        if not os.path.exists(path):
            continue
        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
        print(f'For index {index} count {len(files)} files')
        for file in files:
            raw_image = cv2.imread(f'{path}/{file}', cv2.CV_16U).astype(np.float32)
            raw_image = cv2.resize(raw_image, (W, H))
            raw_image = np.repeat(raw_image[..., np.newaxis], 3, -1)
            raw_image = raw_image.reshape(-1, W, H, 3)

            global max_value_for_normalization
            if np.amax(raw_image) > max_value_for_normalization:
                max_value_for_normalization = np.amax(raw_image)

            new_data = {
                'image': raw_image,
                'Live weight (kg)': df.iloc[index - 3]['Live weight (kg)'],
                'Body length (cm)': df.iloc[index - 3]['Body length (cm)'],
                'Chest girth (cm)': df.iloc[index - 3]['Chest girth (cm)'],
                'Withers height (cm)': df.iloc[index - 3]['Withers height (cm)'],
                'Chest depth (cm)': df.iloc[index - 3]['Chest depth (cm)'],
                'Chest width (cm)': df.iloc[index - 3]['Chest width (cm)'],
            }

            df = pd.concat([df, pd.DataFrame.from_records([new_data])])

    df['image'] = df['image'] / max_value_for_normalization
    df = df.iloc[16:]
    df = df.reset_index().drop(columns=['index'])
    return df
  
dataframe = make_df('./drive/MyDrive/data/data.xlsx', 'mrdata')
dataframe


For index 3 count 31 files
For index 4 count 32 files
For index 5 count 21 files
For index 6 count 31 files
For index 7 count 26 files
For index 8 count 39 files
For index 9 count 27 files
For index 10 count 27 files
For index 11 count 24 files
For index 12 count 27 files
For index 13 count 17 files
For index 14 count 37 files
For index 15 count 22 files
For index 16 count 33 files
For index 17 count 22 files


,Live weight (kg),Body length (cm),Chest girth (cm),Withers height (cm),Chest depth (cm),Chest width (cm),image
0,117,114,117,55,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
1,117,114,117,55,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0.8091..."
2,117,114,117,55,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0.840375 0.840375 ..."
3,117,114,117,55,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
4,117,114,117,55,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
...,...,...,...,...,...,...,...
411,118,123,117,63,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
412,118,123,117,63,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."
413,118,123,117,63,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0.67475 0.67475 0...."
414,118,123,117,63,37,32,"[[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ..."


In [5]:
columns_features_all = ['Live weight (kg)', 'Body length (cm)', 'Chest girth (cm)', 'Withers height (cm)', 'Chest depth (cm)', 'Chest width (cm)']
columns_features_kg = ['Live weight (kg)']
columns_features_sm = ['Body length (cm)', 'Chest girth (cm)', 'Withers height (cm)', 'Chest depth (cm)', 'Chest width (cm)']
# column_img = ['image']

In [6]:
xc_nn = Xception(input_shape=(W, H, 3), include_top=False, weights='imagenet')
xc_nn.trainable = False
# xc_nn.summary()

83697664/83683744 [==============================] - 0s 0us/step


ADAM ALL PARAMS

In [ ]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):

  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_all].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_all].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(xc_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(6, activation='relu'))
  model.build(input_shape=(None, W, H, 3))
  model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=4, verbose=0)
  
  !cp -av ./model/best '/content/drive/MyDrive/data/models/xcp_nn/best_adam_all_params_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 631.14807, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss did not improve from 631.14807

Epoch 00003: val_loss improved from 631.14807 to 443.63849, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00004: val_loss did not improve from 443.63849

Epoch 00005: val_loss did not improve from 443.63849

Epoch 00006: val_loss did not improve from 443.63849

Epoch 00007: val_loss did not improve from 443.63849

Epoch 00008: val_loss improved from 443.63849 to 273.42905, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00009: val_loss did not improve from 273.42905

Epoch 00010: val_loss did not improve from 273.42905

Epoch 00011: val_loss did not improve from 273.42905

Epoch 00012: val_loss improved from 273.42905

In [ ]:
evals_models

[[219.2342987060547, 10.310379028320312, 29.488971710205078],
 [225.5455322265625, 10.738192558288574, 30.65372657775879],
 [37.30366134643555, 4.411095142364502, 12.543574333190918],
 [42.917938232421875, 4.5179762840271, 16.18202781677246],
 [101.31684112548828, 8.350439071655273, 16.09228515625],
 [110.54724884033203, 8.74030876159668, 21.338422775268555],
 [85.6773910522461, 7.297340393066406, 16.400249481201172],
 [88.65817260742188, 7.277186870574951, 18.165964126586914]]

RMSPROP ALL PARAMS

In [ ]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):

  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_all].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_all].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(xc_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(6, activation='relu'))
  model.build(input_shape=(None, W, H, 3))
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=4, verbose=0)
  
  !cp -av ./model/best '/content/drive/MyDrive/data/models/xcp_nn/best_rmsprop_all_params_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 1554.98962, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss improved from 1554.98962 to 1179.26208, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00003: val_loss improved from 1179.26208 to 873.67499, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00004: val_loss improved from 873.67499 to 818.11670, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00005: val_loss improved from 818.11670 to 539.70203, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00006: val_loss improved from 539.70203 to 443.54227, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00007: val_loss did not improve 

In [ ]:
evals_models

[[390.0989074707031, 14.732057571411133, 37.63007736206055],
 [385.4391174316406, 14.301557540893555, 37.25835418701172],
 [365.0924072265625, 15.143275260925293, 36.19097900390625],
 [348.4915771484375, 14.613978385925293, 35.68095779418945],
 [42.294708251953125, 4.804326057434082, 12.044601440429688],
 [54.90193176269531, 5.16709566116333, 17.866092681884766],
 [81.01311492919922, 7.133882999420166, 15.734155654907227],
 [86.08475494384766, 7.4830002784729, 17.91642189025879]]

ADAM KG PARAM

In [ ]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):

  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_kg].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_kg].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(xc_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='relu'))
  model.build(input_shape=(None, W, H, 3))
  model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=4, verbose=0)
  
  !cp -av ./model/best '/content/drive/MyDrive/data/models/xcp_nn/best_adam_kg_param_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 1243.77197, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss improved from 1243.77197 to 557.16925, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00003: val_loss did not improve from 557.16925

Epoch 00004: val_loss improved from 557.16925 to 211.75047, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00005: val_loss did not improve from 211.75047

Epoch 00006: val_loss improved from 211.75047 to 172.39610, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00007: val_loss did not improve from 172.39610

Epoch 00008: val_loss did not improve from 172.39610

Epoch 00009: val_loss did not improve from 172.39610

Epoch 00010: val_loss did not improve from 172.39610

Epoch 

In [ ]:
evals_models

[[35.33500289916992, 4.700753688812256, 4.340552806854248],
 [53.30521011352539, 5.423450946807861, 4.910767555236816],
 [86.91543579101562, 7.814792156219482, 7.388822078704834],
 [89.26622772216797, 8.092939376831055, 7.586465835571289],
 [47.59640884399414, 5.487523555755615, 5.094613552093506],
 [48.218997955322266, 5.5408101081848145, 5.099781036376953],
 [77.34364318847656, 7.070898056030273, 6.502889156341553],
 [53.989479064941406, 5.765279769897461, 5.3500494956970215]]

RMSPROP KG PARAM

In [7]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):

  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_kg].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_kg].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(xc_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='relu'))
  model.build(input_shape=(None, W, H, 3))
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=4, verbose=0)
  
  !cp -av ./model/best '/content/drive/MyDrive/data/models/xcp_nn/best_rmsprop_kg_param_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 11570.32812, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss improved from 11570.32812 to 185.32857, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00003: val_loss did not improve from 185.32857

Epoch 00004: val_loss improved from 185.32857 to 167.81760, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00005: val_loss did not improve from 167.81760

Epoch 00006: val_loss improved from 167.81760 to 147.02989, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00007: val_loss did not improve from 147.02989

Epoch 00008: val_loss improved from 147.02989 to 100.97379, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00009: val_loss

In [8]:
evals_models

[[43.3004035949707, 5.071595668792725, 4.666968822479248],
 [55.13810729980469, 5.928104400634766, 5.427083969116211],
 [205.9256134033203, 12.22372055053711, 11.531034469604492],
 [182.7413787841797, 11.374213218688965, 10.699348449707031],
 [58.471397399902344, 6.036012172698975, 5.641157627105713],
 [70.13882446289062, 6.552180767059326, 6.050386905670166],
 [54.9806022644043, 6.142444133758545, 5.787308216094971],
 [60.29764175415039, 6.379877090454102, 6.047263145446777]]

ADAM SM PARAMS

In [9]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):

  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_sm].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_sm].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(xc_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(5, activation='relu'))
  model.build(input_shape=(None, W, H, 3))
  model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=4, verbose=0)
  
  !cp -av ./model/best '/content/drive/MyDrive/data/models/xcp_nn/best_adam_sm_params_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 665.03247, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss improved from 665.03247 to 290.39120, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00003: val_loss improved from 290.39120 to 252.14214, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00004: val_loss did not improve from 252.14214

Epoch 00005: val_loss did not improve from 252.14214

Epoch 00006: val_loss improved from 252.14214 to 146.44559, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00007: val_loss did not improve from 146.44559

Epoch 00008: val_loss did not improve from 146.44559

Epoch 00009: val_loss did not improve from 146.44559

Epoch 00010: val_loss did not improve from 146.44559

Epoch 00

In [10]:
evals_models

[[101.70560455322266, 7.735497951507568, 17.711524963378906],
 [105.55037689208984, 7.979126930236816, 18.880149841308594],
 [507.6934814453125, 18.416748046875, 46.36066818237305],
 [500.2436828613281, 18.329805374145508, 46.29584503173828],
 [108.53085327148438, 8.482921600341797, 18.667930603027344],
 [121.79721069335938, 9.081001281738281, 25.337791442871094],
 [49.37491226196289, 5.291358947753906, 15.339205741882324],
 [49.81260299682617, 5.27678918838501, 17.702646255493164]]

RMSPROP SM PARAMS

In [11]:
!rm -r './model/best'

count = 1
evals_models = []

ss = ShuffleSplit(n_splits=4, test_size=0.2, random_state=0)
for train_ind, valid_ind in ss.split(dataframe):

  path = './model/best'
  es = EarlyStopping(monitor='val_loss', verbose=1, min_delta=0.02, patience=6)
  mc = ModelCheckpoint(path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  print("Split dataframe")
  y_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind][columns_features_sm].values])
  X_train = np.asarray([np.asarray(i) for i in dataframe.loc[train_ind]['image'].values]).reshape(-1, W, H, 3)
  
  y_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind][columns_features_sm].values])
  X_valid = np.asarray([np.asarray(i) for i in dataframe.loc[valid_ind]['image'].values]).reshape(-1, W, H, 3)

  print("Create model")
  model = Sequential()
  model.add(xc_nn)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(5, activation='relu'))
  model.build(input_shape=(None, W, H, 3))
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae', 'mape'])

  print("Fit")
  history = model.fit(X_train, y_train, epochs=40, validation_split=0.2, validation_data=(X_valid, y_valid), callbacks=[es, mc], batch_size=4, verbose=0)
  
  !cp -av ./model/best '/content/drive/MyDrive/data/models/xcp_nn/best_rmsprop_sm_params_{count}'
  count = count + 1

  print("Evaluate")
  model = keras.models.load_model(path)
  score = model.evaluate(X_train, y_train, verbose=1)
  evals_models.append(score)
  score = model.evaluate(X_valid, y_valid, verbose=1)
  evals_models.append(score)

  !rm -r './model/best'

rm: cannot remove './model/best': No such file or directory
Split dataframe
Create model
Fit

Epoch 00001: val_loss improved from inf to 1776.00708, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00002: val_loss did not improve from 1776.00708

Epoch 00003: val_loss improved from 1776.00708 to 848.97791, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00004: val_loss improved from 848.97791 to 470.22260, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00005: val_loss did not improve from 470.22260

Epoch 00006: val_loss did not improve from 470.22260

Epoch 00007: val_loss did not improve from 470.22260

Epoch 00008: val_loss improved from 470.22260 to 326.81970, saving model to ./model/best
INFO:tensorflow:Assets written to: ./model/best/assets

Epoch 00009: val_loss did not improve from 326.81970

Epoch 00010: val_loss did not improve from 326.81970

Epoch

In [12]:
evals_models

[[299.5027770996094, 12.471210479736328, 27.817909240722656],
 [298.4451904296875, 12.438302040100098, 27.772001266479492],
 [137.74256896972656, 9.449851036071777, 19.518339157104492],
 [157.08282470703125, 10.332710266113281, 24.31820297241211],
 [474.174072265625, 17.16705894470215, 44.83006286621094],
 [451.2811584472656, 16.35971450805664, 44.455352783203125],
 [36.2915153503418, 4.193376541137695, 13.834836959838867],
 [40.050445556640625, 4.310149669647217, 16.46390151977539]]